In [ ]:
# initilize basic setup

from io import StringIO
import numpy as np
import scipy.ndimage as nd
import PIL.Image
from PIL import Image
from IPython.display import clear_output, Image, display
from google.protobuf import text_format
from matplotlib import pyplot as plt
from matplotlib import cm
import glob
%matplotlib inline
import caffe

# comment out below for cpu computing
caffe.set_mode_gpu()
caffe.set_device(0)

import cv2

def showarray(a, fmt='jpeg'):
    a = np.uint8(a)
    f = StringIO()
    PIL.Image.fromarray(a)
    display(PIL.Image.fromarray(a))
        
model_path = 'models/bvlc_googlenet/'
net_fn   = model_path + 'deploy.prototxt'
param_fn = model_path + 'bvlc_googlenet.caffemodel'

model = caffe.io.caffe_pb2.NetParameter()
text_format.Merge(open(net_fn).read(), model)
model.force_backward = True
open('tmp.prototxt', 'w').write(str(model))

net = caffe.Classifier('tmp.prototxt', param_fn,
                       mean = np.float32([104.0, 116.0, 122.0]),
                       channel_swap = (2,1,0))


def preprocess(net, img):
    return np.float32(np.rollaxis(img, 2)[::-1]) - net.transformer.mean['data']

def deprocess(net, img):
    return np.dstack((img + net.transformer.mean['data'])[::-1])

def make_step(net, step_size=1.5, end='inception_4c/output', jitter=32, clip=True):
    '''Basic gradient ascent step.'''

    src = net.blobs['data']
    dst = net.blobs[end]

    ox, oy = np.random.randint(-jitter, jitter+1, 2)
    src.data[0] = np.roll(np.roll(src.data[0], ox, -1), oy, -2)
            
    net.forward(end=end)
    dst.diff[:] = dst.data
    net.backward(start=end)
    g = src.diff[0]
    src.data[:] += step_size/np.abs(g).mean() * g

    src.data[0] = np.roll(np.roll(src.data[0], -ox, -1), -oy, -2)            
    if clip:
        bias = net.transformer.mean['data']
        src.data[:] = np.clip(src.data, -bias, 255-bias)
        
def morphPicture(filename1, filename2, blend, width):
    img1 = PIL.Image.open(filename1)
    img2 = PIL.Image.open(filename2)
    if width is not 0:
        img2 = resizePicture(filename2, width)
    return PIL.Image.blend(img1, img2, blend)

In [ ]:
#initilize deepdream

def deepdream(net, base_img, iter_n=15, octave_n=5, octave_scale=1.4, end='inception_4c/output', clip=True, show_img=False, **step_params):
    octaves = [preprocess(net, base_img)]
    for i in range(octave_n-1):
        octaves.append(nd.zoom(octaves[-1], (1, 1.0/octave_scale,1.0/octave_scale), order=1))
    
    src = net.blobs['data']
    detail = np.zeros_like(octaves[-1])
    for octave, octave_base in enumerate(octaves[::-1]):
        h, w = octave_base.shape[-2:]
        if octave > 0:
            h1, w1 = detail.shape[-2:]
            detail = nd.zoom(detail, (1, 1.0*h/h1,1.0*w/w1), order=1)

        src.reshape(1,3,h,w)
        src.data[0] = octave_base+detail
        for i in range(iter_n):
            make_step(net, end=end, clip=clip, **step_params)
            
            vis = deprocess(net, src.data[0])
            if not clip:
                vis = vis*(255.0/np.percentile(vis, 99.98))
            if show_img:
                showarray(vis)
            print(octave, i, end, vis.shape)
            clear_output(wait=True)       
        detail = src.data[0]-octave_base
        if not show_img:
            showarray(vis)
    return deprocess(net, src.data[0])

def fast_deepdream(net, base_img, iter_n=5, octave_n=5, octave_scale=1.4, end='inception_4c/output', **step_params):
    octaves = [preprocess(net, base_img)]
    for i in range(octave_n-1):
        octaves.append(nd.zoom(octaves[-1], (1, 1.0/octave_scale,1.0/octave_scale), order=1))
    
    src = net.blobs['data']
    detail = np.zeros_like(octaves[-1])
    for octave, octave_base in enumerate(octaves[::-1]):
        h, w = octave_base.shape[-2:]
        if octave > 0:
            h1, w1 = detail.shape[-2:]
            detail = nd.zoom(detail, (1, 1.0*h/h1,1.0*w/w1), order=1)

        src.reshape(1,3,h,w)
        src.data[0] = octave_base+detail
        for i in range(iter_n):
            make_step(net, end=end, clip=True, **step_params)
            
            vis = deprocess(net, src.data[0])
            clear_output(wait=True)
            
        detail = src.data[0]-octave_base
    return deprocess(net, src.data[0])

In [ ]:
# video to frames
vidcap = cv2.VideoCapture('25fpsyunus.mp4')
success,image = vidcap.read()
index = 1
while success:
    cv2.imwrite("./input/%04d.jpg" % index, image)
    success, image = vidcap.read()
    index += 1
    
print('Finished!')

In [ ]:
img = np.float32(PIL.Image.open('input/0001.jpg'))
showarray(img)

In [ ]:
# finding out best settings for deepdream
iter, octave = 4, 3

_ = deepdream(net, img, iter_n=iter, octave_n=octave, clip=True, show_img=True)

In [ ]:
# deepdream frames
blendstatic = 0.1

img_arr = glob.glob('input/*.jpg')
img_arr = sorted(img_arr)
img_i = 1

np_img = np.float32(PIL.Image.open(img_arr[0]))
h, w, c = np_img.shape
frame = fast_deepdream(net, np_img, iter_n=iter, octave_n=octave)
np.clip(frame, 0, 255, out=frame)
gray_img = cv2.cvtColor(np_img, cv2.COLOR_RGB2GRAY)
PIL.Image.fromarray(np.uint8(frame)).save("output/%04d.jpg"%img_i)

for img in img_arr[1:]:
    img_i += 1
    np_prev_img = np_img
    gray_prev_img = gray_img
    
    np_img = np.float32(PIL.Image.open(img))
    gray_img = cv2.cvtColor(np_img, cv2.COLOR_RGB2GRAY)
    
    flow = cv2.calcOpticalFlowFarneback(gray_prev_img, gray_img, None, pyr_scale=0.5, levels=3, winsize=15, iterations=3, poly_n=5, poly_sigma=1.2, flags=0)
    inv_flow = flow
    flow = -flow
    flow[:, :, 0] += np.arange(w)
    flow[:, :, 1] += np.arange(h)[:, np.newaxis]
    
    framediff = morphPicture(np_img, frame, 0.9, 0) - np_prev_img
    framediff = cv2.remap(framediff, flow, None, cv2.INTER_LINEAR)
    frame_flow = np_img + framediff
    
    magnitude, angle = cv2.cartToPolar(inv_flow[...,0], inv_flow[...,1])
    norm_mag = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX);
    ret, mask = cv2.threshold(norm_mag, 6, 255, cv2.THRESH_BINARY);
    flow_mask = mask.astype(np.uint8).reshape((h, w, 1))
    frame_flow_masked = cv2.bitwise_and(frame_flow, frame_flow, mask=flow_mask)
    
    background_blendimg = cv2.addWeighted(np_img, (1-blendstatic), frame, blendstatic, 0)
    background_masked =  cv2.bitwise_and(background_blendimg, background_blendimg, mask=cv2.bitwise_not(flow_mask))
    
    frame = frame_flow_masked + background_masked
    frame = fast_deepdream(net, frame, iter_n=iter, octave_n=octave)
    np.clip(frame, 0, 255, out=frame)
    PIL.Image.fromarray(np.uint8(frame)).save("output/%04d.jpg"%img_i)
    
    print("output/%04d.jpg"%img_i)

In [ ]:
# generate deepdreamed video
fps = 25
frameSize = (640, 360)

mp4_fourcc = cv2.VideoWriter_fourcc(*'MP4V')
avi_fourcc = cv2.VideoWriter_fourcc(*'XVID')

out = cv2.VideoWriter('output_video.avi', avi_fourcc, fps, frameSize)
for filename in sorted(glob.glob('output/*.jpg')):
    img = cv2.imread(filename)
    out.write(img)

out.release()

In [ ]:
# delete everything in input and output file.
import os

def delete_all(pathtodelete):
    files = glob.glob(pathtodelete + '/*')
    for f in files:
        os.remove(f)

delete_all('input')
delete_all('output')